In [1]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)
api_key=os.environ.get("openrouter_api_key")

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model = "qwen/qwen3-14b:free", # "qwen/qwen3-14b:free", "qwen/qwen3-30b-a3b:free",
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    )
llm
# from langchain_groq import ChatGroq
# llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0,max_tokens=3000,) # "gemma2-9b-it", qwen-qwq-32b
# llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002631CEE3950>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002631CF02240>, root_client=<openai.OpenAI object at 0x000002631CCF16A0>, root_async_client=<openai.AsyncOpenAI object at 0x000002631CEE39B0>, model_name='qwen/qwen3-14b:free', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1')

In [3]:
from typing import Union, List
from langchain_ollama import OllamaEmbeddings
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy

def load_elastic_vectorstore(index_names: Union[str, List[str]]):
    # 단일 문자열인 경우 리스트로 변환
    if isinstance(index_names, str):
        index_names = [index_names]
    
    vector_store = ElasticsearchStore(
        index_name=index_names, 
        embedding=OllamaEmbeddings(
            base_url="http://localhost:11434", 
            model="bge-m3:latest"
        ), 
        es_url="http://localhost:9200",
        es_user="Kstyle",
        es_password="12345",
    )
    return vector_store


index_names = ["ship_safety"]
vector_store = load_elastic_vectorstore(index_names=index_names)
vector_store

In [ ]:
from FlagEmbedding import FlagReranker
reranking_model_path = "D:/LLMs/bge-reranker-v2-m3"
reranker = FlagReranker(model_name_or_path=reranking_model_path, 
                        use_fp16=True,
                        batch_size=512,
                        max_length=2048,
                        normalize=True)
reranker

d:\AI_PROJECT\quick_test\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Planning

In [5]:
from typing import Annotated, List, TypedDict, Literal
from pydantic import BaseModel, Field
import operator

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query for web search.")

class Queries(BaseModel):
    queries: List[SearchQuery] = Field(description="List of search queries.",)

class Section(BaseModel):
    name: str = Field(description="Name for this section of the report.",)
    description: str = Field(description="Brief overview of the main topics and concepts to be covered in this section.",)
    research: bool = Field(description="Whether to perform web research for this section of the report.")
    content: str = Field(description="The content of the section.")   

class ReportState(TypedDict):
    topic: str # Report topic    
    feedback_on_report_plan: str # Feedback on the report plan
    sections: list[Section] # List of report sections 
    completed_sections: Annotated[list, operator.add] # Send() API key
    report_sections_from_research: str # String of any completed sections from research to write final sections
    final_report: str # Final report
    index_names:list
    k:int

In [6]:
report_planner_query_writer_instructions="""당신은 보고서를 위한 조사를 수행하고 있습니다.. 

<Report topic>
{topic}
</Report topic>

<Report organization>
{report_organization}
</Report organization>

<Task>
당신의 목표는 보고서의 각 섹션을 계획하는 데 도움이 될 수 있는 웹 검색 쿼리 {number_of_queries}개를 생성하는 것입니다.
생성된 쿼리는 다음 조건을 만족해야 합니다:

    - 보고서 주제와 관련이 있어야 합니다.
    - 보고서 구성에 명시된 요구사항을 충족하는 데 도움이 되어야 합니다.

검색 쿼리는 고품질의 관련 자료를 찾을 수 있을 정도로 구체적이어야 하며, 동시에 보고서 구조 전반을 아우를 수 있도록 폭넓게 다루어야 합니다.
</Task>

<Format>
Queries 도구를 호출하세요
</Format>
"""


DEFAULT_REPORT_STRUCTURE = """다음 구조를 사용하여 사용자가 제공한 주제에 대한 보고서를 작성하세요:

1. 서론 (조사 불필요)

    - 주제 영역에 대한 간략한 개요

2. 본론

    - 각 본론 항목은 사용자가 제공한 주제의 하위 주제에 초점을 맞춰야 함

3. 결론

    - 본론 내용을 요약하는 구조적 요소 1개 포함 (목록 또는 표 중 하나)
    - 보고서의 간결한 요약 제공"""

topic = "최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책"

number_of_queries = 2

In [7]:
system_instructions_query = report_planner_query_writer_instructions.format(topic=topic, report_organization=DEFAULT_REPORT_STRUCTURE, number_of_queries=number_of_queries)
print(system_instructions_query)

당신은 보고서를 위한 조사를 수행하고 있습니다.. 

<Report topic>
최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책
</Report topic>

<Report organization>
다음 구조를 사용하여 사용자가 제공한 주제에 대한 보고서를 작성하세요:

1. 서론 (조사 불필요)

    - 주제 영역에 대한 간략한 개요

2. 본론

    - 각 본론 항목은 사용자가 제공한 주제의 하위 주제에 초점을 맞춰야 함

3. 결론

    - 본론 내용을 요약하는 구조적 요소 1개 포함 (목록 또는 표 중 하나)
    - 보고서의 간결한 요약 제공
</Report organization>

<Task>
당신의 목표는 보고서의 각 섹션을 계획하는 데 도움이 될 수 있는 웹 검색 쿼리 2개를 생성하는 것입니다.
생성된 쿼리는 다음 조건을 만족해야 합니다:

    - 보고서 주제와 관련이 있어야 합니다.
    - 보고서 구성에 명시된 요구사항을 충족하는 데 도움이 되어야 합니다.

검색 쿼리는 고품질의 관련 자료를 찾을 수 있을 정도로 구체적이어야 하며, 동시에 보고서 구조 전반을 아우를 수 있도록 폭넓게 다루어야 합니다.
</Task>

<Format>
Queries 도구를 호출하세요
</Format>



In [8]:
structured_llm = llm.with_structured_output(Queries)
structured_llm

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002631CEE3950>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002631CF02240>, root_client=<openai.OpenAI object at 0x000002631CCF16A0>, root_async_client=<openai.AsyncOpenAI object at 0x000002631CEE39B0>, model_name='qwen/qwen3-14b:free', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1'), kwargs={'response_format': <class '__main__.Queries'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'Queries', 'description': '', 'parameters': {'properties': {'queries': {'description': 'List of search queries.', 'items': {'properties': {'search_query': {'default': None, 'description': 'Query for web search.', 'type': 'string'}}, 'type': 'object'}, 'type': 'array'}}, 'required': ['queries'], 'type':

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage
results = structured_llm.invoke([SystemMessage(content=system_instructions_query),
                                 HumanMessage(content="보고서의 섹션을 계획하는 데 도움이 되는 검색 쿼리를 생성합니다.")])
print(results)

ValidationError: 1 validation error for Queries
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='Queries\n1. "최근 중... 법규 개선 사례"', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

In [103]:
query_list = [query.search_query for query in results.queries]
query_list

['최근 판례 동향과 중대재해 예방 대책의 연관성', '사고 유형별 중대재해 예방 전략']

In [104]:
def retrieve(queries:list):
    retriever = vector_store.as_retriever(
        search_type="mmr", 
        search_kwargs={"fetch_k": 30, "k":5},
        )
    documents = []
    for query in queries:
        docs = retriever.invoke(query)
        documents.extend(docs)    
    return documents

In [105]:
docs = retrieve(queries=query_list)
len(docs)

10

In [127]:
import heapq

def reranking(query: str, docs: list, min_score: float = 0.5, top_k: int = 3):
    """
    doc string
    """
    global reranker
    inputs = [[query, doc.page_content.lower()] for doc in docs]
    scores = reranker.compute_score(inputs)
    if not isinstance(scores, list):
        scores = [scores]

    print(f">>> original scores: {scores}")

    # Filter scores by threshold and keep index
    filtered_scores = [(score, idx) for idx, score in enumerate(scores) if score >= min_score]

    # Get top_k using heapq (more efficient than sorting full list)
    top_scores = heapq.nlargest(top_k, filtered_scores, key=lambda x: x[0])

    # Get document objects from top indices
    reranked_docs = [docs[idx] for _, idx in top_scores]

    return top_scores, reranked_docs


In [108]:
top_scores = []
reranked_docs = []
for query in query_list:
    top_score, documents = reranking(query=query, docs=docs, min_score = 0.05, top_k= 3)
    reranked_docs.extend(documents)
    top_scores.extend(top_score)
print(top_scores)
len(reranked_docs)

>>> original scores: [0.006653281828820394, 0.004334661912783455, 0.02103078762514779, 0.05483313214143513, 0.00012892460232945567, 0.002763947485228386, 0.05483313214143513, 0.007442262554114702, 0.006653281828820394, 0.03453818147148281]
>>> original scores: [0.006129601134295901, 0.00027303826449828495, 0.17000544231342868, 0.0025281418893836746, 0.0016058558107803894, 0.06908743972126688, 0.0025281418893836746, 0.051175549468702924, 0.006129601134295901, 0.4038722786789893]


5

In [109]:
def formatting_docs(docs:list):
    return "\n\n".join(doc.page_content for doc in docs)

source_str = formatting_docs(docs=reranked_docs)
source_str

'This page explains 대법원 2023. 12. 28. 선고 2023도12316 판결 that belongs to 안전 and  판례 categories.\n<!-- image -->\n\n따라서 중대재해처벌법 위반(산업재해치사)죄와 업무상과실치사죄 역시 행위의 동일성이 인정되어 상상적 경 합 관계에 있다 .\n\n나 . 결국 원심이 피고인들에 대한 쟁점 공소사실 부분을 상상적 경합 관계로 판단한 것은 정당하고 , 거기에 상고이유 주장과 같이 죄수 판단에 관한 법리를 오해하여 판결에 영향을 미친 잘못이 없다 . 3 . 나머지 부분에 관한 판단\n\n검사는 피고인들에 대한 원심판결 전부에 대하여 상고하였으나 , 안전조치 및 보건조치 불이행으로 인한 산업 안전보건법 위반 부분에 관하여는 상고장이나 상고이유서에 구체적인 불복이유의 기재가 없다 .\n\n## 4 . 결론\n\n상고를 모두 기각하기로 하여 , 관여 대법관의 일치된 의견으로 주문과 같이 판결한다 .\n\n## 대법관 노정희(재판장) 안철상 이흥구 오석준(주심)\n\nThis page explains 대법원 2023. 12. 28. 선고 2023도12316 판결 that belongs to 안전 and  판례 categories.\n<!-- image -->\n\n따라서 중대재해처벌법 위반(산업재해치사)죄와 업무상과실치사죄 역시 행위의 동일성이 인정되어 상상적 경 합 관계에 있다 .\n\n나 . 결국 원심이 피고인들에 대한 쟁점 공소사실 부분을 상상적 경합 관계로 판단한 것은 정당하고 , 거기에 상고이유 주장과 같이 죄수 판단에 관한 법리를 오해하여 판결에 영향을 미친 잘못이 없다 . 3 . 나머지 부분에 관한 판단\n\n검사는 피고인들에 대한 원심판결 전부에 대하여 상고하였으나 , 안전조치 및 보건조치 불이행으로 인한 산업 안전보건법 위반 부분에 관하여는 상고장이나 상고이유서에 구체적인 불복이유의 기재가 없다 .\n\n## 4 . 결론\n\n상고를 모두 기각하기로 하여 , 관여 대법관의 일치

In [110]:
report_planner_instructions="""간결하고 핵심에 집중된 보고서 계획을 원합니다.

<Report topic>
보고서의 주제는 다음과 같습니다:
{topic}
</Report topic>

<Report organization>
보고서는 다음과 같은 구성에 따라야 합니다:
{report_organization}
</Report organization>

<Context>
보고서의 섹션을 계획할 때 사용할 문맥은 다음과 같습니다:
{context}
</Context>

<Task>
보고서의 섹션 목록을 작성하세요. 계획은 명확하고 집중되어야 하며, 겹치는 섹션이나 불필요한 내용은 없어야 합니다.

예를 들어, 좋은 보고서 구조는 다음과 같을 수 있습니다:
1/ 서론
2/ 주제 A 개요
3/ 주제 B 개요
4/ A와 B의 비교
5/ 결론

각 섹션은 다음 항목을 포함해야 합니다:

    - 이름 : 해당 보고서 섹션의 이름
    - 설명 : 이 섹션에서 다룰 주요 주제에 대한 간략한 개요
    - 조사 : 이 섹션을 위해 웹 조사가 필요한지 여부. 중요: 본문 섹션(서론/결론 제외)은 반드시 조사=True 이어야 함. 보고서는 유용하려면 최소 2~3개의 조사=True 섹션이 있어야 함.
    - 내용 : 섹션의 실제 내용. 지금은 비워둠

통합 지침:

    - 예시와 구현 세부사항은 별도 섹션이 아니라 주요 주제 섹션 내에 포함할 것
    - 각 섹션은 명확히 구분되는 목적을 가져야 하며, 내용이 중복되지 않아야 함
    - 관련 개념은 분리하지 말고 통합할 것
    - 매우 중요: 모든 섹션은 반드시 주제와 직접적인 관련이 있어야 함
    - 주제와 직접 연결되지 않은 주변적인 내용이나 느슨하게 관련된 섹션은 피할 것

제출 전에 구조를 검토하여 중복되는 섹션이 없고 논리적인 흐름을 따르는지 확인하세요.
</Task>

<Feedback>
검토 과정에서 받은 보고서 구조에 대한 피드백이 있다면 다음과 같습니다:
{feedback}
</Feedback>

<Format>
Sections 도구를 호출하세요.
</Format>
"""

In [111]:
system_instructions_sections = report_planner_instructions.format(topic=topic, report_organization=DEFAULT_REPORT_STRUCTURE, context=source_str, feedback="")
print(system_instructions_sections)

간결하고 핵심에 집중된 보고서 계획을 원합니다.

<Report topic>
보고서의 주제는 다음과 같습니다:
최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책
</Report topic>

<Report organization>
보고서는 다음과 같은 구성에 따라야 합니다:
다음 구조를 사용하여 사용자가 제공한 주제에 대한 보고서를 작성하세요:

1. 서론 (조사 불필요)

    - 주제 영역에 대한 간략한 개요

2. 본론

    - 각 본론 항목은 사용자가 제공한 주제의 하위 주제에 초점을 맞춰야 함

3. 결론

    - 본론 내용을 요약하는 구조적 요소 1개 포함 (목록 또는 표 중 하나)
    - 보고서의 간결한 요약 제공
</Report organization>

<Context>
보고서의 섹션을 계획할 때 사용할 문맥은 다음과 같습니다:
This page explains 대법원 2023. 12. 28. 선고 2023도12316 판결 that belongs to 안전 and  판례 categories.
<!-- image -->

따라서 중대재해처벌법 위반(산업재해치사)죄와 업무상과실치사죄 역시 행위의 동일성이 인정되어 상상적 경 합 관계에 있다 .

나 . 결국 원심이 피고인들에 대한 쟁점 공소사실 부분을 상상적 경합 관계로 판단한 것은 정당하고 , 거기에 상고이유 주장과 같이 죄수 판단에 관한 법리를 오해하여 판결에 영향을 미친 잘못이 없다 . 3 . 나머지 부분에 관한 판단

검사는 피고인들에 대한 원심판결 전부에 대하여 상고하였으나 , 안전조치 및 보건조치 불이행으로 인한 산업 안전보건법 위반 부분에 관하여는 상고장이나 상고이유서에 구체적인 불복이유의 기재가 없다 .

## 4 . 결론

상고를 모두 기각하기로 하여 , 관여 대법관의 일치된 의견으로 주문과 같이 판결한다 .

## 대법관 노정희(재판장) 안철상 이흥구 오석준(주심)

This page explains 대법원 2023. 12. 28. 선고 2023도12316 판

In [112]:
planner_message = """보고서의 섹션을 생성합니다. 응답에는 섹션 목록이 포함된 '섹션' 필드가 포함되어야 합니다.
각 섹션에는 이름, 설명, 계획, 연구 및 콘텐츠 필드가 있어야 합니다."""

In [113]:
class Sections(BaseModel):
    sections: List[Section] = Field(description="Sections of the report.",)
    
structured_llm = llm.with_structured_output(Sections)
structured_llm

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CCD2684620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CCD26D5970>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=3000), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Sections', 'description': '', 'parameters': {'properties': {'sections': {'description': 'Sections of the report.', 'items': {'properties': {'name': {'description': 'Name for this section of the report.', 'type': 'string'}, 'description': {'description': 'Brief overview of the main topics and concepts to be covered in this section.', 'type': 'string'}, 'research': {'description': 'Whether to perform web research for this section of the report.', 'type': 'boolean'}, 'content': {'description': 'The content of the section.', 'type': 'string'}}, 'required': ['name', 'description', 'research', 'conte

In [114]:
report_sections = structured_llm.invoke([SystemMessage(content=system_instructions_sections),
                                         HumanMessage(content=planner_message)])
report_sections

Sections(sections=[Section(name='서론', description='최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책에 대한 간략한 개요', research=False, content='최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책에 대한 간략한 개요'), Section(name='산업재해 사례', description='산업재해 사례 및 예방 대책', research=True, content='산업재해 사례 및 예방 대책에 대한 설명'), Section(name='중대재해 예방 대책', description='중대재해 예방을 위한 대책', research=True, content='중대재해 예방을 위한 대책에 대한 설명'), Section(name='결론', description='중대재해 예방 대책의 요약', research=False, content='중대재해 예방 대책의 요약')])

In [132]:
sections = report_sections.sections
sections

[Section(name='서론', description='최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책에 대한 간략한 개요', research=False, content='최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책에 대한 간략한 개요'),
 Section(name='산업재해 사례', description='산업재해 사례 및 예방 대책', research=True, content='산업재해 사례 및 예방 대책에 대한 설명'),
 Section(name='중대재해 예방 대책', description='중대재해 예방을 위한 대책', research=True, content='중대재해 예방을 위한 대책에 대한 설명'),
 Section(name='결론', description='중대재해 예방 대책의 요약', research=False, content='중대재해 예방 대책의 요약')]

In [133]:
# query_list = [query.search_query for query in queries.queries]
# query_list

In [134]:
section_name1 = [sec.name for sec in sections]
print(len(section_name1), section_name1)
section_description1 = [sec.description for sec in sections]
print(len(section_description1), section_description1)
section_research1 = [sec.research for sec in sections]
print(len(section_research1), section_research1)
section_content1 = [sec.content for sec in sections]
print(len(section_content1), section_content1)

4 ['서론', '산업재해 사례', '중대재해 예방 대책', '결론']
4 ['최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책에 대한 간략한 개요', '산업재해 사례 및 예방 대책', '중대재해 예방을 위한 대책', '중대재해 예방 대책의 요약']
4 [False, True, True, False]
4 ['최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책에 대한 간략한 개요', '산업재해 사례 및 예방 대책에 대한 설명', '중대재해 예방을 위한 대책에 대한 설명', '중대재해 예방 대책의 요약']


# Human Feedback (생략)

# Generate Quaries

In [119]:
structured_llm = llm.with_structured_output(Queries)
structured_llm

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CCD2684620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CCD26D5970>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=3000), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Queries', 'description': '', 'parameters': {'properties': {'queries': {'description': 'List of search queries.', 'items': {'properties': {'search_query': {'default': None, 'description': 'Query for web search.', 'type': 'string'}}, 'type': 'object'}, 'type': 'array'}}, 'required': ['queries'], 'type': 'object'}}}], 'ls_structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'name': 'Queries', 'description': '', 'parameters': {'properties': {'queries': {'description': 'List of search queries.', 'items': {'properties': {'search_query': {'d

In [184]:
query_writer_instructions="""당신은 안전관리 기술 보고서의 한 섹션을 작성하기 위해 포괄적인 정보를 수집할 수 있는 웹 검색 쿼리를 생성하는 안전관리 전문가 입니다.

<Report topic>
{topic}
</Report topic>

<Section topic>
{section_topic}
</Section topic>

<Task>
당신의 목표는 위의 섹션 주제에 대한 포괄적인 정보를 수집하는 데 도움이 되는 검색 쿼리 {number_of_queries}개를 생성하는 것입니다.

검색 쿼리는 다음 기준을 충족해야 합니다:

    1. 주제와 관련이 있어야 합니다
    2. 주제의 다양한 측면을 조사해야 합니다

쿼리는 고품질의 관련 있는 자료를 찾을 수 있을 만큼 구체적이어야 합니다.
</Task>

<Format>
Queries 도구를 호출하십시오
</Format>
"""

In [185]:
section_description1[1]

'산업재해 사례 및 예방 대책'

In [186]:
system_instructions = query_writer_instructions.format(topic=topic, 
                                                       section_topic=section_description1[1], 
                                                       number_of_queries=number_of_queries)
print(system_instructions[:200])

당신은 안전관리 기술 보고서의 한 섹션을 작성하기 위해 포괄적인 정보를 수집할 수 있는 웹 검색 쿼리를 생성하는 안전관리 전문가 입니다.

<Report topic>
최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책
</Report topic>

<Section topic>
산업재해 사례 및 예방 대책
</Section topic>

<Task>



In [187]:
from langchain_core.messages import HumanMessage, SystemMessage
queries = structured_llm.invoke([SystemMessage(content=system_instructions),
                                 HumanMessage(content="제공된 주제에 대한 검색 쿼리를 생성합니다")])
queries

Queries(queries=[SearchQuery(search_query='산업재해 사례 및 예방 대책 최근 동향'), SearchQuery(search_query='중대재해 예방을 위한 산업재해 유형별 대책')])

In [188]:
query_list = [query.search_query for query in queries.queries]
query_list

['산업재해 사례 및 예방 대책 최근 동향', '중대재해 예방을 위한 산업재해 유형별 대책']

# Main Retrieve

In [189]:
docs = retrieve(queries=query_list)
len(docs)

10

In [190]:
top_scores = []
reranked_docs = []
for query in query_list:
    top_score, documents = reranking(query=query, docs=docs, min_score = 0.1, top_k= 3)
    reranked_docs.extend(documents)
    top_scores.extend(top_score)
print(top_scores)
len(reranked_docs)

>>> original scores: [0.07767468681990461, 0.4919251835756359, 0.5195376919423713, 0.10466694954337682, 0.5458475251134323, 0.08032510835907902, 0.0015645407490389976, 0.5195376919423713, 0.5527143453616232, 0.01374597891949556]
>>> original scores: [0.4489549085731471, 0.2225055636027022, 0.28447909144292943, 0.03998916058292742, 0.72106105121019, 0.6429020434375288, 0.009082856865999916, 0.28447909144292943, 0.6263811002974002, 0.02057737569306581]
[(0.5527143453616232, 8), (0.5458475251134323, 4), (0.5195376919423713, 2), (0.72106105121019, 4), (0.6429020434375288, 5), (0.6263811002974002, 8)]


6

In [191]:
def formatting_docs(docs:list):
    return "\n\n".join(doc.page_content for doc in docs)

source_str = formatting_docs(docs=docs)
print(source_str[:300])

This page explains 안전보건 실무길잡이_선박건조 및 수리업 that belongs to 안전 and  가이드 categories.
<!-- image -->

## Check Box

## 용어의 정의

- 사업주는 산업재해 조사표에 근로자 대표의 확인올 받아야 하여, 그 기재 내용에 대하여
- 산업재해가 발생한 경우 다음 사항울 기록하고 3년간 보존한다.
- 사업장의 개요 및 근로자의 인적사항, 재해 발생의 일시 및 장소 재해 발생의 원인 및 과정, 재해 재발방지 계획
- 하는 경우 갈음활 수있음

## 중대재해




# Section Write

In [216]:
section_writer_instructions = """안전관리 관련 연구 보고서의 한 부분이 되는 섹션을 작성하십시오.

<Task>
1. 보고서 주제, 섹션 이름, 섹션 주제를 주의 깊게 검토하십시오.
2. 기존 섹션 내용이 있다면 이를 검토하십시오.
3. 제공된 자료(Source material)를 확인하십시오.
4. 보고서 섹션 작성을 위해 사용할 자료를 결정하십시오.
5. 보고서 섹션을 작성하고 사용한 출처를 나열하십시오.
</Task>

<Writing Guidelines>
- 기존 섹션 내용이 비어 있다면 새로 작성하십시오.
- 기존 섹션 내용이 있을 경우, 제공된 자료와 통합하여 작성하십시오.
- 분량은 반드시 300~500단어 이내로 제한하십시오.
- 문장은 간단하고 명확하게 작성하십시오.
- 제공된 자료에 사례가 포함된 경우, 사례의 내용을 최대한 많이 포함해주세요.
- 각 문장의 끝에 근거 문서의 정보를 표기해주세요.(파일제목 및 페이지 번호 등)
- 단락은 짧게 유지하십시오 (최대 7문장).
- 섹션 제목은 Markdown 형식으로 ##를 사용하십시오.
- 섹션 제목을 반복하지 마세요.
</Writing Guidelines>

<Citation Rules>
- 섹션 내용 작성에 직접적으로 관련된 각 고유한 filename에 하나의 인용 번호를 부여하십시오.
- 섹션 끝에는 ### 출처라는 제목 아래 출처 목록을 작성하십시오.
- 중요: 어떤 자료를 선택했든, 최종 목록의 출처 번호는 반드시 빠짐없이 순차적으로 매겨야 합니다 (1,2,3,4…).

- 예시 형식:
  [1] 파일 제목 (page 번호를 알 수 있는 경우, Page 번호 포함)
  [2] 파일 제목  (page 번호를 알 수 있는 경우, Page 번호 포함)
</Citation Rules>

<Final Check>
1. 모든 주장들이 제공된 자료에 기반하고 있는지 확인하십시오.
2. 각 filename이 출처 목록에 단 한 번만 나타나는지 확인하십시오.
3. 출처 번호가 빠짐없이 순차적으로 매겨졌는지 확인하십시오 (1,2,3...).
</Final Check>
"""

section_writer_inputs=""" 
<Report topic>
{topic}
</Report topic>

<Section name>
{section_name}
</Section name>

<Section topic>
{section_topic}
</Section topic>

<Existing section content (if populated)>
{section_content}
</Existing section content>

<Source material>
{context}
</Source material>
"""

In [217]:
print(topic)
num = 1
print(section_name1[num])
print(section_description1[num])
print(source_str[:300])
print(section_content1[num])

최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책
산업재해 사례
산업재해 사례 및 예방 대책
This page explains 안전보건 실무길잡이_선박건조 및 수리업 that belongs to 안전 and  가이드 categories.
<!-- image -->

## Check Box

## 용어의 정의

- 사업주는 산업재해 조사표에 근로자 대표의 확인올 받아야 하여, 그 기재 내용에 대하여
- 산업재해가 발생한 경우 다음 사항울 기록하고 3년간 보존한다.
- 사업장의 개요 및 근로자의 인적사항, 재해 발생의 일시 및 장소 재해 발생의 원인 및 과정, 재해 재발방지 계획
- 하는 경우 갈음활 수있음

## 중대재해


산업재해 사례 및 예방 대책에 대한 설명


In [218]:
section_writer_inputs_formatted = section_writer_inputs.format(topic=topic, 
                                                             section_name=section_name1[num], 
                                                             section_topic=section_description1[num], 
                                                             context=source_str, 
                                                             section_content=section_content1[num])
print(section_writer_inputs_formatted)

 
<Report topic>
최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책
</Report topic>

<Section name>
산업재해 사례
</Section name>

<Section topic>
산업재해 사례 및 예방 대책
</Section topic>

<Existing section content (if populated)>
산업재해 사례 및 예방 대책에 대한 설명
</Existing section content>

<Source material>
This page explains 안전보건 실무길잡이_선박건조 및 수리업 that belongs to 안전 and  가이드 categories.
<!-- image -->

## Check Box

## 용어의 정의

- 사업주는 산업재해 조사표에 근로자 대표의 확인올 받아야 하여, 그 기재 내용에 대하여
- 산업재해가 발생한 경우 다음 사항울 기록하고 3년간 보존한다.
- 사업장의 개요 및 근로자의 인적사항, 재해 발생의 일시 및 장소 재해 발생의 원인 및 과정, 재해 재발방지 계획
- 하는 경우 갈음활 수있음

## 중대재해

0 사망자가 1명 이상 발생한 재해   0 3개월 이상의 요양이 필요한 부상자가 동시에 2명 이상 발생한 재해 부상자 또는 직업성 질병자가 동시에 10명 이상 발생한 재해름 말한다.

## 지체없이

정당한 사유재해 등으로 인한 통신수단의 이용이 곤란하거나 재해자 응급구호; 2차 재해 재발방지루 위한 조치

## 3일 이상의 휴업

(휴업일수에 사고 발생일은 포함되지 않으나 법정공휴일 또는 근로 제공 의무가 없는 휴무일은 포함)

## 작업중지 요건

0 고용노동부장관은 . (중락).. 해당 사업장에 산재가 다시 발생할 급박한 위험이 있다고 판단 되는 경우 그 작업의 중지름 명할 수 있음

- 중대재해가 발생한 해당 직업 중대재해가 발생한 작업과 동일한 작업

0고용노동부장관은 토사 구축물의봉과 화재 폭발 유해하거나 위험한 물질의 누출 등으로 인해 중대재해가 발

In [219]:
section_content = llm.invoke([SystemMessage(content=section_writer_instructions),
                                           HumanMessage(content=section_writer_inputs_formatted)])
section_content

AIMessage(content='## 산업재해 사례\n산업재해 사례 및 예방 대책에 대한 설명은 다양한 산업 분야에서 발생할 수 있는 사고와 그에 따른 피해를 줄이기 위한 대책을 포함합니다. 본 섹션에서는 산업재해 사례와 예방 대책에 대해 자세히 살펴보겠습니다.\n\n산업재해는 작업 중에 발생하는 사고로 인해 근로자가 사망하거나 부상을 입는 경우를 말합니다. 이러한 사고는 작업 환경, 작업자들의 안전 의식, 작업 방법 등 다양한 요인에 의해 발생할 수 있습니다. 따라서 산업재해를 예방하기 위해서는 작업 환경을 안전하게 유지하고, 작업자들이 안전한 작업 방법을 준수하도록 교육하고, 작업 중에 발생할 수 있는 위험을 예측하여 대책을 마련하는 것이 중요합니다.\n\n예를 들어, 선박 수리를 위한 용접 작업 중 바닥으로 떨어지는 사고는 작업 발판 등 근로자가 추락할 위험이 있는 장소에서 발생할 수 있습니다. 이러한 사고를 예방하기 위해서는 작업 발판 등에 안전 난간 등을 설치하고, 근로자에게 안전대를 착용하도록 하는 등 추락 위험을 방지하기 위한 조치를 취해야 합니다.\n\n또한, 리프트 수리 작업 중 와이어로프와 소켓이 리프트 1층 카 상부에서 작업 중이던 근로자의 머리에 부딪혀 사망하는 사고도 발생할 수 있습니다. 이러한 사고를 예방하기 위해서는 리프트 와이어로프 교체 작업 시 상·하부 병행작업을 금지하고, 작업지휘자 선임 및 출입 금지구역 설정, 낙하물 방호선반 설치, 와이어로프 교체 작업에 대한 위험성 평가 실시 및 표준 안전 작업 절차 수립 등이 필요합니다.\n\n산업재해 예방을 위해선 작업자들의 안전 의식과 작업 환경의 안전性을 높이는 것이 중요합니다. 이를 위해 작업자들에게 안전 교육을 제공하고, 작업 환경을 정기적으로 점검하여 위험 요소를 제거하는 것이 필요합니다. 또한, 산업재해 발생 시 즉시 대응하고, 사고의 원인을 분석하여 재발 방지를 위한 대책을 마련하는 것도 중요합니다.\n\n### 출처\n[1] 안전보건 실무길잡이_선박건조 및 수리업 \n[

In [220]:
section_content_str = section_content.content
print(section_content_str)

## 산업재해 사례
산업재해 사례 및 예방 대책에 대한 설명은 다양한 산업 분야에서 발생할 수 있는 사고와 그에 따른 피해를 줄이기 위한 대책을 포함합니다. 본 섹션에서는 산업재해 사례와 예방 대책에 대해 자세히 살펴보겠습니다.

산업재해는 작업 중에 발생하는 사고로 인해 근로자가 사망하거나 부상을 입는 경우를 말합니다. 이러한 사고는 작업 환경, 작업자들의 안전 의식, 작업 방법 등 다양한 요인에 의해 발생할 수 있습니다. 따라서 산업재해를 예방하기 위해서는 작업 환경을 안전하게 유지하고, 작업자들이 안전한 작업 방법을 준수하도록 교육하고, 작업 중에 발생할 수 있는 위험을 예측하여 대책을 마련하는 것이 중요합니다.

예를 들어, 선박 수리를 위한 용접 작업 중 바닥으로 떨어지는 사고는 작업 발판 등 근로자가 추락할 위험이 있는 장소에서 발생할 수 있습니다. 이러한 사고를 예방하기 위해서는 작업 발판 등에 안전 난간 등을 설치하고, 근로자에게 안전대를 착용하도록 하는 등 추락 위험을 방지하기 위한 조치를 취해야 합니다.

또한, 리프트 수리 작업 중 와이어로프와 소켓이 리프트 1층 카 상부에서 작업 중이던 근로자의 머리에 부딪혀 사망하는 사고도 발생할 수 있습니다. 이러한 사고를 예방하기 위해서는 리프트 와이어로프 교체 작업 시 상·하부 병행작업을 금지하고, 작업지휘자 선임 및 출입 금지구역 설정, 낙하물 방호선반 설치, 와이어로프 교체 작업에 대한 위험성 평가 실시 및 표준 안전 작업 절차 수립 등이 필요합니다.

산업재해 예방을 위해선 작업자들의 안전 의식과 작업 환경의 안전性을 높이는 것이 중요합니다. 이를 위해 작업자들에게 안전 교육을 제공하고, 작업 환경을 정기적으로 점검하여 위험 요소를 제거하는 것이 필요합니다. 또한, 산업재해 발생 시 즉시 대응하고, 사고의 원인을 분석하여 재발 방지를 위한 대책을 마련하는 것도 중요합니다.

### 출처
[1] 안전보건 실무길잡이_선박건조 및 수리업 
[2] 사고사례_8 
[3] 사고사례_1 
[4] 사고사례_

# Grade

In [221]:
section_grader_instructions = """보고서의 특정 주제에 해당하는 섹션을 검토하세요:

<Report topic>
{topic}
</Report topic>

<section topic>
{section_topic}
</section topic>

<section content>
{section}
</section content>

<task>
섹션 내용이 섹션 주제를 충분히 다루고 있는지 평가하세요.
섹션 내용이 섹션 주제를 충분히 다루지 못한 경우, 부족한 정보를 수집하기 위한 검색 질의 {number_of_follow_up_queries}개를 생성하세요.
</task>

<format>
Feedback 도구를 호출하고 다음 스키마에 따라 출력하세요:

grade: Literal["pass","fail"] = Field(
    설명="응답이 요구사항을 충족하는지('pass') 또는 수정이 필요한지('fail')를 나타내는 평가 결과."
)
follow_up_queries: List[SearchQuery] = Field(
    설명="후속 검색 질의 목록."
)
</format>
"""

In [222]:
section_grader_message = ("보고서를 평가하고 누락된 정보에 대한 후속 질문을 고려합니다."
                          "보고서 평가결과가 'pass'인 경우 모든 후속 쿼리에 빈 문자열을 반환합니다."
                          "보고서 평가결과가 'fail'인 경우 누락된 정보를 수집하기 위해 특정 검색 쿼리를 제공합니다."
                         )

In [223]:
section_grader_instructions_formatted = section_grader_instructions.format(topic=topic, 
                                                                            section_topic=section_description1[num],
                                                                            section=section_content1[num], 
                                                                            number_of_follow_up_queries=number_of_queries)
section_grader_instructions_formatted

'보고서의 특정 주제에 해당하는 섹션을 검토하세요:\n\n<Report topic>\n최근 판례 동향을 고려한, 사고 유형별 중대재해 예방 대책\n</Report topic>\n\n<section topic>\n산업재해 사례 및 예방 대책\n</section topic>\n\n<section content>\n산업재해 사례 및 예방 대책에 대한 설명\n</section content>\n\n<task>\n섹션 내용이 섹션 주제를 충분히 다루고 있는지 평가하세요.\n섹션 내용이 섹션 주제를 충분히 다루지 못한 경우, 부족한 정보를 수집하기 위한 검색 질의 2개를 생성하세요.\n</task>\n\n<format>\nFeedback 도구를 호출하고 다음 스키마에 따라 출력하세요:\n\ngrade: Literal["pass","fail"] = Field(\n    설명="응답이 요구사항을 충족하는지(\'pass\') 또는 수정이 필요한지(\'fail\')를 나타내는 평가 결과."\n)\nfollow_up_queries: List[SearchQuery] = Field(\n    설명="후속 검색 질의 목록."\n)\n</format>\n'

In [224]:
class Feedback(BaseModel):
    grade: Literal["pass","fail"] = Field(description="Evaluation result indicating whether the response meets requirements ('pass') or needs revision ('fail').")
    follow_up_queries: List[SearchQuery] = Field(description="List of follow-up search queries.",)

In [225]:
reflection_model = llm.with_structured_output(Feedback)
reflection_model

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CCD2684620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CCD26D5970>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=3000), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Feedback', 'description': '', 'parameters': {'properties': {'grade': {'description': "Evaluation result indicating whether the response meets requirements ('pass') or needs revision ('fail').", 'enum': ['pass', 'fail'], 'type': 'string'}, 'follow_up_queries': {'description': 'List of follow-up search queries.', 'items': {'properties': {'search_query': {'default': None, 'description': 'Query for web search.', 'type': 'string'}}, 'type': 'object'}, 'type': 'array'}}, 'required': ['grade', 'follow_up_queries'], 'type': 'object'}}}], 'ls_structured_output_format': {'kwargs': {'method': 'function_ca

In [226]:
feedback = reflection_model.invoke([SystemMessage(content=section_grader_instructions_formatted),
                                    HumanMessage(content=section_grader_message)])
feedback

Feedback(grade='fail', follow_up_queries=[SearchQuery(search_query='산업재해 사례 및 예방 대책의 최신 동향'), SearchQuery(search_query='중대재해 예방을 위한 산업재해 유형별 대책')])

In [227]:
feedback.grade

'fail'

In [228]:
new_quary_list = feedback.follow_up_queries
new_quary_list

[SearchQuery(search_query='산업재해 사례 및 예방 대책의 최신 동향'),
 SearchQuery(search_query='중대재해 예방을 위한 산업재해 유형별 대책')]